In [ ]:
"""



Performed so badly that it wasn't even worth including in the presentation or report

Leaving it here because I can

I will not be commenting or cleaning up any of it



"""

In [21]:
"""
Run once ever

Downloads and extracts the dataset
"""


# import urllib.request
# import tarfile
# import os

# mp3file = urllib.request.urlopen("https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz")
# with open('tmp','wb') as output:
#   output.write(mp3file.read())

# file = tarfile.open("tmp")
# file.extractall(".")
# file.close()
# os.remove("tmp")
# os.remove("cifar-100-python/file.txt~")

'\nRun once ever\n\nDownloads and extracts the dataset\n'

In [22]:

import pickle
import pandas as pd
import numpy as np
%matplotlib inline
import cv2
from collections import Counter
import threading
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
#function to read files present in the Python version of the dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

In [24]:
trainData = unpickle('./cifar-100-python/train')
#type of items in each file
for item in trainData:
    print(item, type(trainData[item]))

filenames <class 'list'>
batch_label <class 'str'>
fine_labels <class 'list'>
coarse_labels <class 'list'>
data <class 'numpy.ndarray'>


In [25]:
testData = unpickle('./cifar-100-python//test')
metaData = unpickle('./cifar-100-python//meta')
#metaData
print("Fine labels:", metaData['fine_label_names'], "\n")
print("Coarse labels:", metaData['coarse_label_names'])

Fine labels: ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', '

In [26]:
#storing fine labels along with its number code in a dataframe
subCategory = pd.DataFrame(metaData['fine_label_names'], columns=['SubClass'])
print(subCategory)

         SubClass
0           apple
1   aquarium_fish
2            baby
3            bear
4          beaver
..            ...
95          whale
96    willow_tree
97           wolf
98          woman
99           worm

[100 rows x 1 columns]


In [27]:
# I'm pretty sure this is set up so that dimension 0 is the category
X_train = trainData['data']
X_train = X_train.reshape(len(X_train),3,32,32).transpose(0,2,3,1)

In [28]:
X_test = testData['data']
X_test = X_test.reshape(len(X_test),3,32,32).transpose(0,2,3,1)
#X_test

In [29]:
winSize = (32,32)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (2, 2)
nbins = 12


df = []

**Create Database of HOGs** 
(Pandas dataframe)

In [30]:
# this is essentially where we fit the model
DF = []
for i in range(len(X_train)):
# for i in range(5000):
    image = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    df.append(hog.compute(image))
DF = np.array(df)
df = []
for i in range(len(X_test)):
    image = cv2.cvtColor(X_test[i], cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    df.append(hog.compute(image))
DF_test = np.array(df)

In [31]:
DF[0]

array([0.        , 0.        , 0.00412205, ..., 0.00682359, 0.0028279 ,
       0.06439196], shape=(9408,), dtype=float32)

****continue work checkpoint: standardize test data and proceed****

In [32]:
def predict(img, k, winSize, blockSize, blockStride, cellSize, nbins):
    
    #Extract sample's features
    # sample = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    # _hog = hog.compute(sample)
    
    #Get Nearest Neighbors
    distances = []
    for i in range(len(DF)):
        distances.append(np.linalg.norm(img - X_train[i]))         # 1- Calculate and store each datapoint's Euclidean distance
    # distances = np.linalg.norm((np.expand_dims(_hog, axis=0) - DF))
    
    kNNsID = np.argsort(distances)[:k]                              # 2- Sort distances' IDs in an ascending order
    
    kNNsCats = []
    for i in kNNsID:
        kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs
    
    CommonCat = Counter(kNNsCats).most_common(1)                    # 4- Count and get Most common category
    
    return CommonCat

In [33]:
n = 25
p = predict(X_test[n], 20, winSize, blockSize, blockStride, cellSize, nbins)

/tmp/ipykernel_185303/2195589398.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs


In [34]:
def getAccuracy(k, x_test, testdata, winSize, blockSize, blockStride, cellSize, nbins):
    preds = []
    RightCount = 0
    for i in range(1000):
        pred = predict(x_test[i], k, winSize, blockSize, blockStride, cellSize, nbins)
        if pred[0][0] == subCategory.iloc[testdata['fine_labels'][i]][0].capitalize():
            RightCount += 1
        preds.append(pred)
        print("Done: {}, Accuracy: {}".format(i,(RightCount/(i+1))*100), flush=True)
    # it would be RightCount / 1000 * 100, so I just made it RightCount / 10
    return RightCount / 10

In [35]:
y_train = trainData["fine_labels"]
y_test = testData["fine_labels"]
X_train = trainData["data"]
X_test = testData["data"]

# X_train = np.array(pd.concat([pd.DataFrame(X_train), DF], axis=1))
# X_test = np.array(pd.concat([pd.DataFrame(X_test), DF_test], axis=1))
X_train = np.concat([X_train, DF], axis=1)
X_test = np.concat([X_test, DF_test], axis=1)

print(X_train.shape)
print(X_test.shape)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

(50000, 12480)
(10000, 12480)


In [37]:
# n_components = [5000]


# def do(X_test_scaled):
#     accuracy = getAccuracy(10, X_test_scaled, testData, winSize, (8,8), (4,4), (2,2), 12)
#     print(f"n: {n}, Accuracy: {accuracy}\n\n\n\n\n\n\n\n\n\n\n\n\n")

# threads = []
# for n in n_components:
#     pca = PCA(n_components=n)
#     pca.fit_transform(X_train_scaled)
#     pca.transform(X_test_scaled)
#     # threads.append(threading.Thread(target=do, args=(np.array(X_test_scaled),) ))
#     # threads[len(threads)-1].start()
#     accuracy = getAccuracy(10, X_test_scaled, testData, winSize, (8,8), (4,4), (2,2), 12)
#     print(f"n: {n}, Accuracy: {accuracy}")

lda = LinearDiscriminantAnalysis()
lda.fit_transform(X_train_scaled, y_train)
accuracy = getAccuracy(10, X_test_scaled, testData, winSize, (8,8), (4,4), (2,2), 12)
print(f"Accuracy: {accuracy}")

    
# for t in threads:
#     t.join()

Done: 0, Accuracy: 0.0


/tmp/ipykernel_185303/2195589398.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs
/tmp/ipykernel_185303/703372369.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pred[0][0] == subCategory.iloc[testdata['fine_labels'][i]][0].capitalize():


Done: 1, Accuracy: 0.0
Done: 2, Accuracy: 0.0
Done: 3, Accuracy: 0.0
Done: 4, Accuracy: 0.0
Done: 5, Accuracy: 0.0
Done: 6, Accuracy: 0.0
Done: 7, Accuracy: 0.0
Done: 8, Accuracy: 0.0
Done: 9, Accuracy: 0.0
Done: 10, Accuracy: 0.0
Done: 11, Accuracy: 0.0
Done: 12, Accuracy: 0.0
Done: 13, Accuracy: 0.0
Done: 14, Accuracy: 0.0
Done: 15, Accuracy: 0.0
Done: 16, Accuracy: 0.0
Done: 17, Accuracy: 0.0
Done: 18, Accuracy: 0.0
Done: 19, Accuracy: 0.0
Done: 20, Accuracy: 0.0
Done: 21, Accuracy: 0.0
Done: 22, Accuracy: 0.0
Done: 23, Accuracy: 0.0
Done: 24, Accuracy: 0.0
Done: 25, Accuracy: 0.0
Done: 26, Accuracy: 0.0
Done: 27, Accuracy: 0.0
Done: 28, Accuracy: 0.0
Done: 29, Accuracy: 0.0
Done: 30, Accuracy: 0.0
Done: 31, Accuracy: 0.0
Done: 32, Accuracy: 0.0
Done: 33, Accuracy: 0.0
Done: 34, Accuracy: 0.0
Done: 35, Accuracy: 0.0
Done: 36, Accuracy: 0.0
Done: 37, Accuracy: 0.0
Done: 38, Accuracy: 0.0
Done: 39, Accuracy: 0.0
Done: 40, Accuracy: 0.0
Done: 41, Accuracy: 0.0
Done: 42, Accuracy: 0.0
D

In [ ]:
# print(grid_search.cv_results_)
# print(grid_search.best_params_)

In [ ]:
TruePreds = []

for i in range(1000):
    TruePreds.append(subCategory.iloc[testData['fine_labels'][i]][0].capitalize())

/tmp/ipykernel_3225649/3796635586.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TruePreds.append(subCategory.iloc[testData['fine_labels'][i]][0].capitalize())


In [ ]:
# Predicted = []
# for i in range(len(Preds14)):
#     Predicted.append(Preds14[i][0][0])

In [ ]:
# cm = confusion_matrix(TruePreds, Predicted)

In [ ]:
# x = np.array(TruePreds)
# len(np.unique(x))

In [ ]:
# cm_df = pd.DataFrame(cm,
#                      index = np.unique(x), 
#                      columns = np.unique(x))

In [ ]:
# plt.figure(figsize=(16,16))
# sns.heatmap(cm_df, annot=True)
# plt.title('Confusion Matrix')
# plt.ylabel('Actal Values')
# plt.xlabel('Predicted Values')
# plt.show()